<a href="https://colab.research.google.com/github/ashif-ali/hsfl/blob/main/HSFL_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch flask pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
import torch
import torch.nn as nn

class CoreModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(128, 10),
            nn.Softmax(dim=1)
        )

    def forward(self, z):
        return self.fc(z)


In [3]:
from pyngrok import ngrok

ngrok.set_auth_token("2wfRWx4TE1taWLOq2NyY7qXnMH4_3aqT3FHPmfDzytKaUU7sn")

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import torch
import torch.nn as nn
import threading

app = Flask(__name__)
core_model = CoreModel()
criterion = nn.CrossEntropyLoss()

@app.route('/forward_backward', methods=['POST'])
def forward_backward():
    data = request.json
    smashed_data = torch.tensor(data['smashed_data'], requires_grad=True)
    targets = torch.tensor(data['targets'])

    predictions = core_model(smashed_data)
    loss = criterion(predictions, targets)
    loss.backward()

    gradients = smashed_data.grad.detach().tolist()
    return jsonify({'gradients': gradients})

def run_flask():
    app.run(port=5000)

# Clearly launch Flask app in a separate thread
threading.Thread(target=run_flask).start()

# Ngrok tunnel
public_url = ngrok.connect(5000).public_url
print("Server URL:", public_url)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Server URL: https://fa07-34-105-36-66.ngrok-free.app


In [ ]:
def fed_avg(weight_files):
    avg_weights = torch.load(weight_files[0])
    for key in avg_weights.keys():
        avg_weights[key] = torch.mean(
            torch.stack([torch.load(wf)[key] for wf in weight_files]), dim=0
        )
    return avg_weights

client_weights = [
    '/content/drive/MyDrive/HSFL_Project/client_1/client_1_weights.pt',
    '/content/drive/MyDrive/HSFL_Project/client_2/client_2_weights.pt',
    '/content/drive/MyDrive/HSFL_Project/client_3/client_3_weights.pt'
]

global_weights = fed_avg(client_weights)

# Save averaged weights explicitly to Drive
torch.save(global_weights, '/content/drive/MyDrive/HSFL_Project/server/global_peripheral_weights.pt')
